## 加载向量数据库

首先，我们加载在前一章已经构建的向量数据库。注意，此处你需要使用和构建时相同的 Emedding。

设置embedding - zhipu

In [1]:
# import sys
# sys.path.append("../C3 搭建知识库") # 将父目录放入系统路径中

# 使用智谱 Embedding API，注意，需要将上一章实现的封装代码下载到本地
from zhipuai_embedding import ZhipuAIEmbeddings

from langchain.vectorstores.chroma import Chroma

from dotenv import load_dotenv, find_dotenv
import os
# 从环境变量中加载你的 API_KEY
_ = load_dotenv(find_dotenv())    # read local .env file
zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']

# zhipuai_api_key

# 加载数据库

import os
from dotenv import load_dotenv, find_dotenv
from langchain.vectorstores.chroma import Chroma
from zhipuai_embedding import ZhipuAIEmbeddings  # 假设这是正确的导入路径


_ = load_dotenv(find_dotenv())    # read local .env file
zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']

# 定义持久化目录
persist_directory = '../chromatest'

# 创建嵌入模型
from langchain_community.embeddings import ZhipuAIEmbeddings

zhipu_embed = ZhipuAIEmbeddings(
    model="embedding-2",
    api_key=zhipuai_api_key
)

try:
    # 加载持久化的 Chroma 向量数据库
    vectordb = Chroma(
        persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
        embedding_function=zhipu_embed
    )
    print("向量数据库已成功加载。")
except Exception as e:
    print(f"加载向量数据库时发生错误: {e}")

C:\Users\will\AppData\Local\Temp\ipykernel_12328\2576265592.py:5: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from zhipuai_embedding import ZhipuAIEmbeddings
C:\Users\will\AppData\Local\Temp\ipykernel_12328\2576265592.py:41: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and imp

向量数据库已成功加载。


从环境变量中加载你的 API_KEY

In [ ]:
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(find_dotenv())    # read local .env file
zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']

zhipuai_api_key

加载向量数据库，其中包含了 ../../data_base/knowledge_db 下多个文档的 Embedding

In [ ]:
# # 定义 Embeddings
# # embeddings = ZhipuAIEmbeddings()
# # embeddings = OpenAIEmbeddings()
# embedding = ZhipuAIEmbeddings()

# # 向量数据库持久化路径
# # persist_directory = '../C3 搭建知识库/data_base/vector_db/chroma'
# persist_directory = '../../data_base/vector_db/chroma'
# # 加载数据库
# vectordb = Chroma(
#     persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
#     embedding_function=embedding
# )

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain.vectorstores.chroma import Chroma
from zhipuai_embedding import ZhipuAIEmbeddings  # 假设这是正确的导入路径


_ = load_dotenv(find_dotenv())    # read local .env file
zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']

# 定义持久化目录
persist_directory = '../../data_base/vector_db/chroma'

# 创建嵌入模型
embedding = ZhipuAIEmbeddings()

try:
    # 加载持久化的 Chroma 向量数据库
    vectordb = Chroma(
        persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
        embedding_function=embedding
    )
    print("向量数据库已成功加载。")
except Exception as e:
    print(f"加载向量数据库时发生错误: {e}")

In [ ]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

In [ ]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

In [ ]:
question = "什么是机器学习?"
docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(docs)}")

打印一下检索到的内容

In [ ]:
for i, doc in enumerate(docs):
    print(f"检索到的第{i}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")

## 创建一个LLM

在这里，我们调用 OpenAI 的 API 创建一个 LLM，当然你也可以使用其他 LLM 的 API 进行创建

In [2]:
from langchain_openai import ChatOpenAI
zhipuai_llm = ChatOpenAI(
    temperature=0.95,
    model="glm-4",
    openai_api_key=zhipuai_api_key,
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)

## 进行问答

基于rerank回答

In [3]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank

# Cohere Rerank配置
import cohere
cohere_client = cohere.Client(api_key="Tahx1eySFbKvu9sTyTXrRLf59la3ZUG9vy02stRZ")
compressor = CohereRerank(
    client=cohere_client,
    top_n=5  # 保留Top5相关文档
)

# 创建带重排序的检索器
# base_retriever = vectordb.as_retriever(search_kwargs={"k": 10})
base_retriever = vectordb.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 15, "lambda_mult": 0.5}
)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)

# 5. LLM模型初始化
# from openai import OpenAI 

# llm = OpenAI(
#     api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io",
#     base_url="https://open.bigmodel.cn/api/paas/v4/"
# ) 
from langchain_community.chat_models import ChatOpenAI
llm = ChatOpenAI(
    temperature=0.95,
    model="glm-4",
    openai_api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io",
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)

# 6. 构建RAG链
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 提示模板设计
template = """基于以下上下文，用中文回答用户问题：
上下文：{context}
问题：{question}
要求：答案需包含引用来源的页码（示例：[P3]）"""
prompt = ChatPromptTemplate.from_template(template)

# 组装RAG链
rag_chain = (
    {"context": compression_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

# 7. 使用示例
response = rag_chain.invoke("机器学习")
print(response)

C:\Users\will\AppData\Local\Temp\ipykernel_12328\1207227334.py:7: LangChainDeprecationWarning: The class `CohereRerank` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import CohereRerank``.
  compressor = CohereRerank(
C:\Users\will\AppData\Local\Temp\ipykernel_12328\1207227334.py:27: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


content='根据您提供的上下文，关于“机器学习”的相关信息分布在几页文档中。其中涉及到的内容包括梯度下降法、牛顿法、线性判别分析以及深度学习的概念等。以下是对这些内容的简要概述，并标明相应的页码：\n\n- **梯度下降法**：在文档的第3页，介绍了梯度下降法的概念。[P3]\n- **牛顿法**：在第3页中，也对牛顿法进行了讲解。[P3]\n- **线性判别分析**：同样在第3页，涉及线性判别分析的内容。[P3]\n- **深度学习**：从第4页开始，介绍了深度学习的概念，包括深度学习的起源和特征学习的理解。[P4]\n\n这些内容均来源于南瓜书《机器学习公式详解》。如果需要更详细的信息，请参阅对应的纸质版书籍或电子文档。' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 178, 'prompt_tokens': 1700, 'total_tokens': 1878, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'glm-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-f5f2fa3a-9da1-42a7-93cf-80e5a71cabf2-0'


对比norank 和 rank

In [6]:
# 基础检索器（无重排序）
base_retriever = vectordb.as_retriever(search_kwargs={"k": 10})

# 构建RAG链
basic_rag_chain = (
    {"context": base_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

In [7]:
# 带重排序的检索器
compression_retriever = ContextualCompressionRetriever(
    base_compressor=CohereRerank(client=cohere_client, top_n=5),
    base_retriever=base_retriever
)

# 构建RAG链
rerank_rag_chain = (
    {"context": compression_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

In [8]:
# 测试同一问题
question = "机器学习的主要应用场景有哪些？"

# 无Rerank的结果
basic_response = basic_rag_chain.invoke(question)
print("=== 无Rerank的答案 ===")
print(basic_response)

# 带Rerank的结果
rerank_response = rerank_rag_chain.invoke(question)
print("\n=== 带Rerank的答案 ===")
print(rerank_response)

=== 无Rerank的答案 ===
content='机器学习的主要应用场景包括但不限于以下几个方面：\n\n1. 图像识别与处理：如图像分类、目标检测、图像分割等。[P4]\n2. 自然语言处理：如文本分类、情感分析、机器翻译、语音识别等。[P4]\n3. 推荐系统：根据用户的历史行为和偏好，为用户推荐合适的商品、服务等。[P4]\n4. 数据挖掘：从海量数据中挖掘有价值的信息，如关联规则挖掘、聚类分析等。[P3]\n5. 语音识别与处理：将语音信号转换为文本信息，应用于语音助手、语音翻译等场景。[P3]\n6. 生物信息学：如基因序列分析、蛋白质结构预测等。[P3]\n7. 金融领域：如信用评分、风险管理、股票预测等。[P3]\n\n以上仅是部分应用场景，实际上机器学习在许多其他领域也有广泛的应用。引用来源页码为上述文档中的页码。' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 200, 'prompt_tokens': 3257, 'total_tokens': 3457, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'glm-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-240f27f7-08be-4818-b6e7-e5fb5adbbb63-0'

=== 带Rerank的答案 ===
content='机器学习的主要应用场景包括但不限于以下几个方面：\n\n1. 图像识别与处理：如图像分类、目标检测、图像分割等。[P3]\n2. 自然语言处理：如文本分类、情感分析、机器翻译、语音识别等。[P3]\n3. 推荐系统：如电商推荐、电影推荐、音乐推荐等。[P3]\n4. 数据挖掘：如用户行为分析、异常检测、关联规则挖掘等。[P3]\n5. 语音识别与处理：如语音唤醒、语音助手、语音合成等。[P4]\n6. 深度学习：在各类应用场景中，深度学习技术都取得了显著的成果，如自动驾驶、医疗诊断、机器人等。[P4]\n\n请注意

打印详细记录

In [9]:
# 在原有代码基础上新增以下对比逻辑

# 执行原始检索（无rerank）
base_docs = base_retriever.invoke("机器学习")
print("=== Rerank前检索结果 ===")
for i, doc in enumerate(base_docs):
    print(f"[文档{i+1}] 相似度分：{doc.metadata.get('score', 'N/A')}")
    print(f"内容摘要：{doc.page_content[:100]}...")
    print(f"元数据：页码={doc.metadata.get('page', 'N/A')}, 来源={doc.metadata.get('source', 'N/A')}\n")

# 执行带rerank的检索
reranked_docs = compression_retriever.invoke("机器学习")
print("\n=== Rerank后检索结果 ===")
for i, doc in enumerate(reranked_docs):
    print(f"[文档{i+1}] 重排序分：{doc.metadata.get('relevance_score', 'N/A')}")
    print(f"内容摘要：{doc.page_content[:100]}...")
    print(f"元数据：页码={doc.metadata.get('page', 'N/A')}, 来源={doc.metadata.get('source', 'N/A')}\n")

# 原有RAG流程保持不变
response = rag_chain.invoke("机器学习")
print("\n=== 最终答案 ===")
print(response)

=== Rerank前检索结果 ===
[文档1] 相似度分：N/A
内容摘要：→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》←_←5.5
其他常见神经网络
........................................455.5.1
式(5.1...
元数据：页码=4, 来源=../data_base/knowledge_path\pumkin_book\pumpkin_book.pdf

[文档2] 相似度分：N/A
内容摘要：→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》←_←5.5
其他常见神经网络
........................................455.5.1
式(5.1...
元数据：页码=4, 来源=../data_base/knowledge_path\pumkin_book\pumpkin_book.pdf

[文档3] 相似度分：N/A
内容摘要：→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》←_←5.5
其他常见神经网络
........................................455.5.1
式(5.1...
元数据：页码=4, 来源=../data_base/knowledge_path\pumkin_book\pumpkin_book.pdf

[文档4] 相似度分：N/A
内容摘要：→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》←_←5.5
其他常见神经网络
........................................455.5.1
式(5.1...
元数据：页码=4, 来源=../data_base/knowledge_path\pumkin_book\pumpkin_book.pdf

[文档5] 相似度分：N/A
内容摘要：→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》←_←
目录
第1章绪论
11.1
引言................................................1...
元数据：页码=2, 来源=../data_base/knowledge_path\pumkin_book\pumpkin_book.pdf

[文档6]

In [14]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain_community.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
import cohere

# ----------------- 改进点1：增强元数据处理 -----------------
def format_document_metadata(docs):
    """为文档添加规范化元数据（参考网页1的注释规范）"""
    for doc in docs:
        if 'page' not in doc.metadata:
            doc.metadata['page'] = 'P0'  # 默认页码
        if 'source' not in doc.metadata:
            doc.metadata['source'] = 'unknown'
    return docs

# ----------------- 改进点2：优化检索配置 -----------------
def init_retrievers(vector_db):
    """初始化带/不带rerank的检索器（参考网页6的检索器配置）"""
    # 基础检索器（MMR算法平衡多样性）
    base_retriever = vector_db.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 10, "lambda_mult": 0.3}
    )
    
    # Cohere重排序配置（参考网页8的检索优化）
    cohere_client = cohere.Client(api_key="Tahx1eySFbKvu9sTyTXrRLf59la3ZUG9vy02stRZ")
    compressor = CohereRerank(client=cohere_client, top_n=5)
    
    # 带重排序的检索器
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=base_retriever
    )
    
    return base_retriever, compression_retriever

# ----------------- 改进点3：增强结果可视化 -----------------
def print_retrieval_results(retriever, query, title):
    """打印检索结果（含元数据展示，参考网页4的格式化输出）"""
    docs = retriever.invoke(query)
    formatted_docs = format_document_metadata(docs)
    
    print(f"\n=== {title} ===")
    for i, doc in enumerate(formatted_docs[:3]):  # 展示Top3结果
        print(f"• 结果{i+1} [页码:{doc.metadata['page']}]")
        print(f"  相似度分：{doc.metadata.get('score', 0):.2f}")
        print(f"  内容摘要：{doc.page_content[:80]}...\n")

# ----------------- 主执行流程 -----------------
if __name__ == "__main__":
    # 初始化组件
    base_retriever, rerank_retriever = init_retrievers(vectordb)
    llm = ChatOpenAI(
        model="glm-4",
        temperature=0.7,  # 调整生成稳定性（参考网页5的温度系数建议）
        openai_api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io",
        openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
    )
    
    # 打印检索对比结果
    query = "机器学习"
    print_retrieval_results(base_retriever, query, "原始检索结果（MMR算法）")
    print_retrieval_results(rerank_retriever, query, "重排序后结果（Cohere Rerank）")
    
    # ----------------- 改进点4：增强提示工程 -----------------
    prompt_template = ChatPromptTemplate.from_template("""
    基于以下上下文，用中文回答用户问题：
    [知识来源]
    {context}
    
    [回答要求]
    1. 答案需包含引用来源的页码（示例：[P3]）
    2. 若存在矛盾信息，请说明各来源差异
    3. 使用Markdown格式组织内容
    
    当前问题：{question}
    """)
    
    # 构建RAG链（参考网页7的Runnable组合方式）
    rag_chain = (
        {"context": rerank_retriever, "question": RunnablePassthrough()} 
        | prompt_template 
        | llm
    )
    
    # 执行问答
    print("\n=== 优化后的问答结果 ===")
    print(rag_chain.invoke(query))


=== 原始检索结果（MMR算法） ===
• 结果1 [页码:4]
  相似度分：0.00
  内容摘要：→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》←_←5.5
其他常见神经网络
....................................

• 结果2 [页码:4]
  相似度分：0.00
  内容摘要：→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》←_←5.5
其他常见神经网络
....................................

• 结果3 [页码:4]
  相似度分：0.00
  内容摘要：→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》←_←5.5
其他常见神经网络
....................................


=== 重排序后结果（Cohere Rerank） ===
• 结果1 [页码:3]
  相似度分：0.00
  内容摘要：→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》←_←3.3.1
式(3.27)的推导
................................

• 结果2 [页码:4]
  相似度分：0.00
  内容摘要：→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》←_←5.5
其他常见神经网络
....................................

• 结果3 [页码:4]
  相似度分：0.00
  内容摘要：→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》←_←5.5
其他常见神经网络
....................................


=== 优化后的问答结果 ===
content='根据提供的知识来源，以下是关于机器学习的相关信息：\n\n机器学习的内容包括但不限于梯度下降法、牛顿法、线性判别分析以及深度学习等概念。[P3] \n\n- 梯度下降法在文献中的介绍位于第3.3.2节。[P3]\n- 牛顿法的相关内容则在第3.3.3节。[P3]\n- 线性判别分析的相关推导可以在第3.4节找到。[P3]\n  \n另外，还有关于其他常见神经网络以及深度学习的讨论：\n\n- 在第5.5节中，有对其他常见神经网络的

In [15]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain_community.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
import cohere

# ----------------- Chroma专属配置 -----------------
def chroma_score_converter(docs):
    """将Chroma的距离值转换为相似度分数（余弦距离→相似度）"""
    for doc in docs:
        if 'distances' in doc.metadata:
            # Chroma的余弦距离计算式为 1 - cosine_similarity
            distance = doc.metadata['distances'][0]  # 取第一个距离值
            doc.metadata['score'] = 1 - distance  # 转换为相似度分数
    return docs

# ----------------- 改进点1：增强元数据处理 -----------------
def format_document_metadata(docs):
    """为文档添加规范化元数据"""
    docs = chroma_score_converter(docs)  # 先转换分数
    for doc in docs:
        if 'page' not in doc.metadata:
            doc.metadata['page'] = 'P0'
        if 'source' not in doc.metadata:
            doc.metadata['source'] = 'unknown'
    return docs

# ----------------- 改进点2：优化检索配置 -----------------
def init_retrievers(vector_db):
    """初始化带/不带rerank的检索器（Chroma专属配置）"""
    # Chroma检索器需显式请求距离值
    base_retriever = vector_db.as_retriever(
        search_type="mmr",
        search_kwargs={
            "k": 10,
            "lambda_mult": 0.3,
            "include": ["distances"]  # 关键配置：获取距离信息
        }
    )
    
    # Cohere重排序配置
    cohere_client = cohere.Client(api_key="Tahx1eySFbKvu9sTyTXrRLf59la3ZUG9vy02stRZ")
    compressor = CohereRerank(client=cohere_client, top_n=5)
    
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=base_retriever
    )
    
    return base_retriever, compression_retriever

# ----------------- 改进点3：增强结果可视化 -----------------
def print_retrieval_results(retriever, query, title):
    """打印检索结果（适配Chroma的分数格式）"""
    raw_docs = retriever.invoke(query)
    processed_docs = format_document_metadata(raw_docs)
    
    print(f"\n=== {title} ===")
    for i, doc in enumerate(processed_docs[:3]):
        print(f"• 结果{i+1} [页码:{doc.metadata['page']}]")
        print(f"  向量相似度：{doc.metadata.get('score', 0):.2f}")
        if 'relevance_score' in doc.metadata:
            print(f"  语义重排分：{doc.metadata['relevance_score']:.2f}")
        print(f"  内容摘要：{doc.page_content[:80]}...\n")

# ----------------- 主执行流程 -----------------
if __name__ == "__main__":
    # 初始化组件（假设vectordb已正确初始化）
    base_retriever, rerank_retriever = init_retrievers(vectordb)
    llm = ChatOpenAI(
        model="glm-4",
        temperature=0.7,
        openai_api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io",
        openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
    )
    
    # 打印对比结果
    query = "机器学习"
    print_retrieval_results(base_retriever, query, "原始检索结果（Chroma MMR）")
    print_retrieval_results(rerank_retriever, query, "重排序后结果（Cohere Rerank）")
    
    # 构建RAG链
    prompt_template = ChatPromptTemplate.from_template("""
    基于以下上下文，用中文回答用户问题：
    [知识来源]
    {context}
    
    [回答要求]
    1. 包含引用页码（示例：[P3]）
    2. 冲突信息需说明来源差异
    
    当前问题：{question}
    """)
    
    rag_chain = (
        {"context": rerank_retriever, "question": RunnablePassthrough()} 
        | prompt_template 
        | llm
    )
    
    print("\n=== 优化后的问答结果 ===")
    print(rag_chain.invoke(query))


=== 原始检索结果（Chroma MMR） ===
• 结果1 [页码:4]
  向量相似度：0.00
  内容摘要：→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》←_←5.5
其他常见神经网络
....................................

• 结果2 [页码:4]
  向量相似度：0.00
  内容摘要：→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》←_←5.5
其他常见神经网络
....................................

• 结果3 [页码:4]
  向量相似度：0.00
  内容摘要：→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》←_←5.5
其他常见神经网络
....................................


=== 重排序后结果（Cohere Rerank） ===
• 结果1 [页码:3]
  向量相似度：0.00
  语义重排分：0.98
  内容摘要：→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》←_←3.3.1
式(3.27)的推导
................................

• 结果2 [页码:4]
  向量相似度：0.00
  语义重排分：0.97
  内容摘要：→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》←_←5.5
其他常见神经网络
....................................

• 结果3 [页码:4]
  向量相似度：0.00
  语义重排分：0.97
  内容摘要：→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》←_←5.5
其他常见神经网络
....................................


=== 优化后的问答结果 ===
content='根据提供的上下文信息，关于机器学习的内容可以在南瓜书《机器学习公式详解》中找到。以下是相关信息：\n\n- [P3] 介绍了式(3.27)的推导，梯度下降法，牛顿法，以及式(3.29)、式(3.30)、式(3.31)的解释和推导。\n- [P4] 涉及了其他常见神经网络，式(5.18)、式(5.20)、式(5.22)、式(5.23)